In [1]:
import torch
from dataset.dataloader import EmoDataset
from models.model import MyEmoSubword
import torch.optim as optim
from torch.utils.data import DataLoader

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [3]:
device

device(type='mps')

In [4]:
def main():
    # 假设我们用一个 dict 来存所有参数
    args = {
        'model_path': 'meta-llama/Llama-3.1-8B',
        'file_path': 'data/eng/train.csv',
        'lang': 'en',
        'max_len': 128,
        'num_labels': 5,
        'dropout': 0.1,
        'joint_loss': 'joint',
        'alpha': 0.2,
        'pool_type': 'mean'
    }

    # 构建 Dataset
    dataset = EmoDataset(args)
    batch_size = 4

    dataloader = DataLoader(dataset, batch_size, shuffle=True)

    # 构建 Model
    model = MyEmoSubword(args)
    model.to(device)

    # 简单的训练循环
    optimizer = optim.AdamW(model.parameters(), lr=1e-5)
    model.train()
    for epoch in range(3):
        for batch in dataloader:
            input_ids, labels, label_positions = batch

            # 把数据移动到 MPS
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            label_positions = label_positions.to(device)

            # Forward
            loss, bsz, preds, golds = model((input_ids, labels, label_positions))

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [5]:
main()

<|end_of_text|>


/Users/zhuangxiaojian/Documents/research/panemo/dataset/dataloader.py:99: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  all_labels = torch.tensor(all_labels, dtype=torch.float)                              # [dataset_size, num_labels]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards:  50%|█████     | 2/4 [01:07<01:10, 35.25s/it]

: 

device(type='mps')